In [1]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint, BFGS
import sys
from functools import reduce
from typing import Tuple, List 
np.set_printoptions(threshold=sys.maxsize)

##### Simulation and Optimization Code

In [2]:
# Global hyperparameters
E1, E2, E3 = 1, 2, 3
Ndim = 15   # 1-7 rho for segment 1-6, with 0, 7 dummy segment 0, 7 for rho, 8-13 v for segment 1-7, 14 w_r
T, L, Lambda, D_r = 1 / 360, 1, 2, 1500
Tau, Mu, C_r, Rho_m, Alpha, K, A, V_f, Rho_c = 19, 60, 2000, 120, 0.1, 40, 1.867, 120, 33 + E1 / 3

In [3]:
# Utility method (softmax min)
def soft_min(xs: List, beta: float=50.0):
    nx = np.array(xs)
    m = np.min(nx)
    return m - (1.0 / beta) * np.log(np.sum(np.exp(-beta * (nx - m))))

In [4]:
# Wrap up sophisticated functions in the simulation as well as optimization to produce less bugs
def _observation(states: np.ndarray, i: int):
    return T * states[i, -1] + T * L * Lambda * np.sum(states[i, 1:7])

def _speed_dynamics(states: np.ndarray, control_inputs: np.ndarray, i: int, j: int):
    prv_desired_speed = V_f * np.exp(-np.pow(-soft_min([-states[i-1, j], 0]) / Rho_c, A) / A)
    prv_V = soft_min([(1 + Alpha) * control_inputs[i-1, 0].item(), prv_desired_speed]) if j == 2 or j == 3 else prv_desired_speed
    return states[i-1, j+7] + T / Tau * (prv_V - states[i-1, j+7]) + T / L * states[i-1, j+7] * ((states[i-1, j+6] - states[i-1, j+7]) if j > 1 else 0) - (Mu * T * (states[i-1, j+1] - states[i-1, j])) / (Tau * L * (states[i-1, j] + K))

def _init_density(states: np.ndarray, i: int):
    v_0, cur_q_0 = states[i, 8], (3000 + 50 * E2) if i < 60 else (1000 + 50 * E2)
    return cur_q_0 / (Lambda * v_0)

def _density_dynamics(states: np.ndarray, i: int, j: int):
    return states[i-1, j] + T / L * states[i-1, j-1] * (states[i-1, j-1+7] if j > 1 else states[i-1, 8]) - T / L * states[i-1, j] * states[i-1, j+7]

def _ramp_density(states: np.ndarray, control_inputs: np.ndarray, i: int):
    return soft_min([control_inputs[i-1, 1].item() * C_r, D_r + states[i-1, -1].item() / T, C_r * (Rho_m - states[i-1, 5].item()) / (Rho_m - Rho_c)])

def _add_ramp_up(states: np.ndarray, control_inputs: np.ndarray, i: int):
    return T / (Lambda * L) * _ramp_density(states, control_inputs, i)

def _queue_length_dynamics(states: np.ndarray, control_inputs: np.ndarray, i: int):
    return max(0, states[i-1, -1] + T * (D_r - _ramp_density(states, control_inputs, i)))

In [5]:
def simulate(num_step: int=120, vsl_: np.array=None, r_: np.array=None):
    vsl, r = np.repeat(vsl_, 6), np.repeat(r_, 6)
    assert len(vsl) == num_step
    control_inputs: np.ndarray = np.column_stack((vsl, r))
    states, outputs = np.zeros(shape=(num_step+1, Ndim)), np.zeros(shape=num_step+1)
    # Time boundary conditions
    states[0, 0] = 25
    for i in range(1, 8): states[0, i], states[0, i+7] = 25, 80
    states[0, -1] = 0
    # Time Evolution
    for i in range(1, num_step+1):
        # evaluate speed
        for j in range(1, 7):
            states[i, j+7] = _speed_dynamics(states, control_inputs, i, j)
        # evaluate density
        states[i, 0] = _init_density(states, i)
        for j in range(1, 7):
            states[i, j] = _density_dynamics(states, i, j)
            if j == 5:
                states[i, j] += _add_ramp_up(states, control_inputs, i)
        # evaluate queue size
        states[i, 7] = states[i, 6]
        states[i, -1] = _queue_length_dynamics(states, control_inputs, i)
        # evaluate output
        outputs[i] = _observation(states, i)
    return states, outputs

In [6]:
def unpack(num_step: int, opt_vars: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    controls, states = opt_vars[:2*num_step], opt_vars[2*num_step:]
    return controls.reshape((-1, 2)), states.reshape((-1, Ndim))

def pack(controls: np.ndarray, states: np.ndarray) -> np.ndarray:
    return np.append(controls.reshape(-1), states.reshape(-1))

In [7]:
def optimize(num_step: int=120, init_controls: np.ndarray=None, init_states: np.ndarray=None):
    def objective(opt_vars: np.ndarray):
        assert not np.isnan(opt_vars).any() and not np.isinf(opt_vars).any()
        controls, states = unpack(num_step, opt_vars)
        return reduce(lambda x, y: x + y, map(lambda idx: _observation(states, idx), range(1, num_step+1)))
    
    def dynamics_constraint(opt_vars: np.ndarray):
        """
        Express the nonlinear equality constraints x[k+1]=f(x[k],u[k]) => f(x[k],u[k])-x[k+1]=0
        """
        controls, states = unpack(num_step, opt_vars) 
        results = np.zeros(shape=num_step*Ndim) 
        # Initialization Contraints
        results[0] = states[0, 0] - 25
        for i in range(1, 8):
            results[i] = states[i, 0] - 25
            if i != 7: results[i+7] = states[0, i+7] - 80
        results[14] = states[0, -1]
    
        for i in range(0, num_step):
            # Speed contraints: same with speed dynamics
            for j in range(1, 7):
                results[i*Ndim+j+7] = _speed_dynamics(states, controls, i, j) - states[i, j+7]
            # Initial density constraints: v_0(k) = v_1(k), q_0(k) = lambda * rho_0(k) * v_0(k)
            results[i*Ndim] = _init_density(states, i) - states[i, 0]
            # Density constraints
            for j in range(1, 7):
                results[i*Ndim+j] = _density_dynamics(states, i, j) - states[i, j]
                if j == 5:
                    results[i*Ndim+j] += _add_ramp_up(states, controls, i)
            results[i*Ndim+7] = states[i, 7] - states[i, 6]
            results[(i+1)*Ndim-1] = states[i, -1] - _queue_length_dynamics(states, controls, i)
        return results

    init_opt_vars = pack(init_controls, init_states) # define initial values for optimization
    nonlinear_contraints = NonlinearConstraint(dynamics_constraint, lb=np.zeros(shape=num_step*Ndim), ub=np.zeros(shape=num_step*Ndim), jac='3-point', hess=BFGS())     # define nonlinear contraints
    bounds = [(0, 1), (60, 120)] * num_step + ([(0, Rho_c)] * 8 + [(0, 200)] * 6 + [(0, C_r)])  * (num_step + 1)
    assert len(init_opt_vars) == len(bounds)
    result = minimize(objective, init_opt_vars, method='trust-constr', jac='3-point', hess=BFGS(), bounds=bounds, constraints=[nonlinear_contraints], options={'verbose': 2, "gtol": 1e-6, "xtol": 1e-6, "barrier_tol": 1e-8,})
    print(f"Success: {result.success}, message: {result.message}")
    optimal_opt_vars = result.x
    optimal_controls, optimal_states = unpack(num_step, optimal_opt_vars)
    return optimal_controls, optimal_states

##### Task Code for the problems

In [8]:
def task1(num_control: int):
    vsl_, r_ = np.random.rand(num_control) * 60 + 60, np.random.rand(num_control)
    states, outputs = simulate(num_control * 6, vsl_, r_)
    print(states)
    print(outputs)

In [9]:
def task3():
    init_states, _ = simulate(120, np.zeros(20), np.full(20, 60))
    init_controls = np.array([0, 60] * 120)
    optimize(120, init_controls, init_states)

In [10]:
task3()

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  |
|-------|-------|-------|-------------|----------|----------|----------|
|   1   | 4111  |   0   | +7.3482e+01 | 1.00e+00 | 5.05e-02 | 1.82e+01 |
|   2   | 8222  |   1   | +7.3468e+01 | 7.00e+00 | 5.03e-02 | 1.82e+01 |
|   3   | 12333 |   2   | +7.3364e+01 | 4.90e+01 | 4.95e-02 | 1.64e+01 |
|   4   | 16444 |   5   | +7.1772e+01 | 3.43e+02 | 3.82e-02 | 8.23e+00 |
|   5   | 20555 |   9   | +6.7755e+01 | 5.46e+02 | 2.21e-02 | 4.15e+00 |
|   6   | 24666 |  13   | +6.5147e+01 | 5.46e+02 | 1.66e-02 | 6.19e+00 |
|   7   | 28777 |  17   | +6.1017e+01 | 8.20e+02 | 3.48e-02 | 3.11e+00 |
|   8   | 32888 |  23   | +5.9303e+01 | 8.20e+02 | 2.61e-02 | 1.61e+00 |
|   9   | 41110 |  30   | +5.9303e+01 | 4.10e+02 | 2.61e-02 | 1.61e+00 |
|  10   | 45221 |  34   | +5.9727e+01 | 4.10e+02 | 2.62e-02 | 8.06e+00 |
|  11   | 49332 |  39   | +5.9727e+01 | 1.24e+02 | 2.62e-02 | 8.06e+00 |
|  12   | 53443 |  41   | +5.9727e+01 | 3.79e+01 | 

/home/oreki/.venv/lib/python3.12/site-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)


|  164  |661871 |  263  | +2.9005e+01 | 3.26e+02 | 4.09e-02 | 9.97e-03 |
|  165  |665982 |  518  | +2.9005e+01 | 3.26e+01 | 4.09e-02 | 9.97e-03 |
|  166  |670093 |  773  | +2.9005e+01 | 3.26e+00 | 4.09e-02 | 9.97e-03 |
|  167  |674204 | 1028  | +2.9005e+01 | 3.26e-01 | 4.09e-02 | 9.97e-03 |
|  168  |678315 | 1283  | +2.9005e+01 | 3.26e-02 | 4.09e-02 | 9.97e-03 |
|  169  |682426 | 1284  | +2.9005e+01 | 3.26e-03 | 4.09e-02 | 9.97e-03 |
|  170  |686537 | 1285  | +2.9005e+01 | 3.26e-04 | 4.09e-02 | 9.97e-03 |
|  171  |690648 | 1286  | +2.9005e+01 | 3.26e-05 | 4.09e-02 | 9.97e-03 |
|  172  |694759 | 1287  | +2.9005e+01 | 3.26e-06 | 4.09e-02 | 9.97e-03 |
|  173  |698870 | 1288  | +2.9005e+01 | 3.26e-07 | 4.09e-02 | 9.97e-03 |


/home/oreki/.venv/lib/python3.12/site-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:81: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)


|  174  |702981 | 1288  | +2.9005e+01 | 1.00e+00 | 4.17e-02 | 9.97e-03 |
|  175  |707092 | 1543  | +2.9005e+01 | 1.00e-01 | 4.17e-02 | 9.97e-03 |
|  176  |711203 | 1798  | +2.9005e+01 | 1.00e-02 | 4.17e-02 | 9.97e-03 |


KeyboardInterrupt: 